In [1]:
!pip install openai
import openai
import base64
from bs4 import BeautifulSoup
import requests
from google.colab import userdata
import json

In [2]:
with open('scrapedTextNoImagesTwo.txt', 'r') as file:
  text = file.read()

In [3]:
pages = text.split("\u3019")
# Removes the initial unicode character used to denote the start of a page
pages = [x[1:] for x in pages]

In [4]:
prompt = '''
I am training a large language model (LLM) using information from a wiki.
My goal is to evaluate the model's comprehension, factual recall, and reasoning
abilities by generating a set of specific, detailed questions to test it.

Please avoid vague, broad, or generic questions such as 'What does this text say' or
other general questions that assume broader context. Focus on generating
specific, detailed, and meaningful questions based only on the information
provided in the text. The questions should not assume knowledge outside of the provided
text, nor reference 'this page', 'this section,' 'this text', or other location-based language.

Treat the text as a standalone passage, and generate factual, inferential, or conceptual
questions that can be answered **directly** from the content provided. Avoid
hallucinations and ensure that the questions target specific elements from the text.

Please return a JSON object with the following keys:
1. Keyname: "url". Contains a URL that is found at the beginning of the text I provide for you.
2. Keyname: "Q/A". A Q/A array where each array element is another object in JSON notation
containing a 'question' key that holds a specific, detail-focused question and an 'answer' key
containing the correct answer from the provided text.
3. A 'total' key that lists the total number of questions.

Make sure to use double quotes everywhere, unless you're writing a quote
within a double quote, in which case you can use single quotes.
'''

In [5]:
def questionGeneration(page):
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {userdata.get('OPEN_API_KEY')}"
    }

    payload = {
      "model": "gpt-4o-mini",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": f"{prompt}. Now here is wiki page: {page}"
            },
          ]
        }
      ],
      "max_tokens": 3000
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Prints the response, then reformat's GPTs json string
    print(response.json()['choices'][0]['message']['content'])
    return response.json()['choices'][0]['message']['content'].replace("\n", "")[7:-3]

In [ ]:
with open('questionsList.json', 'a') as file:
    file.write("[")
for i in range(len(pages)):
  print(f"Generating questions about page {i}")
  ans = {}
  for j in range(3):
    try:
        ans = json.loads(questionGeneration(pages[i]))
        break
    except:
        continue

  with open('questionsList.json', 'a') as file:
    json.dump(ans, file, ensure_ascii=False, indent=4)
    file.write(",")
with open('questionsList.json', 'a') as file:
  file.write("]")

Generating questions about page 0
